# Derivation - With pytorch

Besides numpy, we also have torch backend for the main types. This means that if you have a GPU, you can use it for the basic operations!

Please keep in mind that unfortunately, communication related operations do not happen through the GPUs, but by mpi, therefore one needs to use the numpy versions for those operations. Possible parallel suport in the future for torch directly.

#### Import general modules

In [1]:
# Import required modules
from mpi4py import MPI #equivalent to the use of MPI_init() in C
import matplotlib.pyplot as plt
import numpy as np

# Get mpi info
comm = MPI.COMM_WORLD

#### Import modules from pysemtools

In [2]:
from pysemtools.io.ppymech.neksuite import pynekread
from pysemtools.datatypes.msh import Mesh
from pysemtools.datatypes.coef import Coef
from pysemtools.datatypes.field import FieldRegistry

## Read the data and build objects

In this instance, we create connectivity for the mesh object, given that we wish to use direct stiffness summation to reduce discontinuities.

In [ ]:
msh = Mesh(comm, create_connectivity=False, bckend='torch')
fld = FieldRegistry(comm, bckend='torch')
pynekread('../data/mixlay0.f00001', comm, data_dtype=np.double, msh=msh, fld=fld)
coef = Coef(msh, comm, get_area=False)

2025-02-25 23:16:06,043 - Mesh - INFO - Initializing empty Mesh object.
2025-02-25 23:16:06,045 - Field - INFO - Initializing empty Field object
2025-02-25 23:16:06,046 - pynekread - INFO - Reading file: ../data/mixlay0.f00001
2025-02-25 23:16:06,050 - Mesh - INFO - Initializing Mesh object from x,y,z ndarrays.
2025-02-25 23:16:06,050 - Mesh - INFO - Initializing common attributes.
2025-02-25 23:16:06,051 - Mesh - INFO - Getting vertices
2025-02-25 23:16:06,052 - Mesh - INFO - Getting edge centers
2025-02-25 23:16:06,055 - Mesh - INFO - Facet centers not available for 2D
2025-02-25 23:16:07,059 - Mesh - INFO - Mesh object initialized.
2025-02-25 23:16:07,060 - Mesh - INFO - Mesh data is of type: torch.float64
2025-02-25 23:16:07,061 - Mesh - INFO - Elapsed time: 1.011047044s
2025-02-25 23:16:07,061 - pynekread - INFO - Reading field data
2025-02-25 23:16:07,066 - pynekread - INFO - File read
2025-02-25 23:16:07,066 - pynekread - INFO - Elapsed time: 1.019858059s
2025-02-25 23:16:07,067

TypeError: Cannot interpret 'torch.float64' as a data type

In [ ]:
print(fld.fields['vel'][0].device)

cpu


### Plot the 2D velocity field

In [ ]:
fig, ax = plt.subplots(figsize=(5, 2.5), dpi = 200)
c = ax.tricontourf(msh.x.flatten(), msh.y.flatten() ,fld.registry["u"].flatten(), levels=100, cmap="RdBu_r")
fig.colorbar(c)
ax.set_aspect('equal')
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_xlim([1.5,6.5])
ax.set_ylim([5.5,8.5])
plt.show()

## Obtain x and y derivatives of u

Differentiation in physical space is done by using the chain rule. 

We always differentiate the the field in the reference element, i.e., we get: dudr, duds, dudt (in 3d).

To obtain the derivative in physical space, we must simply pass as inputs the components of the inverse of the jacobian matrix that will map the derivatives in the reference element to the direction we want.

For example in this case, to get the derivative in x we need to perform:

dudx = dudr * drdx + duds * dsdx

Therefore:

In [ ]:
dudx = coef.dudxyz(fld.registry['u'], coef.drdx, coef.dsdx)
dudy = coef.dudxyz(fld.registry['u'], coef.drdy, coef.dsdy)

### Plot the derivatives

In [ ]:
fig, ax = plt.subplots(figsize=(5, 2.5), dpi = 200)
c = ax.tricontourf(msh.x.flatten(), msh.y.flatten() ,dudx.flatten(), levels=np.linspace(-2.5,3.1,100), cmap="RdBu_r")
fig.colorbar(c)
ax.set_aspect('equal')
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_xlim([1.5,6.5])
ax.set_ylim([5.5,8.5])
plt.show()

fig, ax = plt.subplots(figsize=(5, 2.5), dpi = 200)
c = ax.tricontourf(msh.x.flatten(), msh.y.flatten() ,dudy.flatten(), levels=np.linspace(-4.6,7.6,100), cmap="RdBu_r")
fig.colorbar(c)
ax.set_aspect('equal')
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_xlim([1.5,6.5])
ax.set_ylim([5.5,8.5])
plt.show()